<a href="https://colab.research.google.com/github/daisysong76/AI--Machine--learning/blob/main/Text_to_SQL_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Natural Language Query Parsing:
Use a pre-trained LLM (e.g., OpenAI GPT or Hugging Face T5) to convert user queries into SQL statements.


Database Schema Understanding:
Dynamically retrieve the schema of the connected database to ensure SQL generation aligns with database structure.


Feedback Loop for Query Optimization:
Implement real-time query evaluation and refinement based on database execution results.


Fine-tuned Model:
Fine-tune a pre-trained model (e.g., T5 or Codex) on a Text-to-SQL dataset like Spider to improve domain-specific query translation.


In [ ]:
pip install transformers psycopg2 pandas openai


In [ ]:
import psycopg2

def get_db_schema(connection):
    query = """
    SELECT table_name, column_name, data_type
    FROM information_schema.columns
    WHERE table_schema = 'public';
    """
    cursor = connection.cursor()
    cursor.execute(query)
    schema = cursor.fetchall()
    cursor.close()
    return schema

# Connect to PostgreSQL
connection = psycopg2.connect(
    dbname="your_database",
    user="your_user",
    password="your_password",
    host="localhost",
    port="5432"
)
schema = get_db_schema(connection)
print("Database Schema:", schema)


In [ ]:
from transformers import pipeline

# Load a pre-trained Text-to-SQL model
generator = pipeline("text2text-generation", model="t5-small")

def generate_sql(query, schema):
    schema_str = "\n".join([f"{table}.{column}" for table, column, _ in schema])
    input_text = f"Schema: {schema_str}\nQuery: {query}\nSQL:"
    sql = generator(input_text, max_length=128, num_return_sequences=1)
    return sql[0]['generated_text']

user_query = "Show me the names of customers who made purchases over $500."
sql_query = generate_sql(user_query, schema)
print("Generated SQL:", sql_query)


In [ ]:
def execute_sql(connection, sql_query):
    cursor = connection.cursor()
    try:
        cursor.execute(sql_query)
        results = cursor.fetchall()
        return results
    except Exception as e:
        print("Error executing query:", e)
    finally:
        cursor.close()

results = execute_sql(connection, sql_query)
print("Query Results:", results)


In [ ]:
def refine_sql(user_query, sql_query, feedback):
    # Append feedback to fine-tuning dataset
    with open("fine_tuning_data.txt", "a") as file:
        file.write(f"Query: {user_query}\nSQL: {sql_query}\nFeedback: {feedback}\n\n")
    print("Feedback saved for refinement.")

refine_sql(user_query, sql_query, "SQL syntax error: column name mismatch.")


In [ ]:
python run_seq2seq.py \
    --model_name_or_path t5-small \
    --do_train \
    --train_file fine_tuning_data.json \
    --output_dir text_to_sql_model \
    --num_train_epochs 3 \
    --per_device_train_batch_size 8


Additional Enhancements
Front-End Interface:
Build a simple UI using Flask or FastAPI to accept user queries and display SQL results.

Context Awareness:
Maintain query context for multi-step user interactions (e.g., follow-up questions).

Caching and Optimization:
Cache frequently executed queries to reduce latency.